In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from utils.plotting_tools import save_fig
from utils.constants import SPEED_OF_SOUND

# A. Basics

In [ ]:
def plot_spectrum(spectrum, polar=True, title="", log=False, ax=None, ylabels=True, 
                  mics_clean=None, sources=None):
    if polar:
        if ax is None:
            fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
        else:
            fig = plt.gcf()
            
        plot_polar(BeamFormer.theta_scan, spectrum, ax, log=log, ylabels=ylabels)
        #plt.polar(BeamFormer.theta_scan, spectrum[0, :])
    else:
        if ax is None:
            fig, ax = plt.subplots()
        else:
            fig = plt.gcf()
        
        if log:
            plt.semilogy(BeamFormer.theta_scan_deg, spectrum[0, :])
        else:
            plt.plot(BeamFormer.theta_scan_deg, spectrum[0, :])
            
    if (mics_clean is not None) and (sources is not None):
        for mics in mics_clean:
            center = np.mean(mics, axis=0)
            for source in sources:
                normal = source - center
                angle = np.arctan2(normal[1], normal[0])
        
                if polar: 
                    ax.axvline(angle, color="k", ls=":")
                else:
                    ax.axvline(angle * 180 / np.pi, color="k", ls=":")
    plt.title(title, y=1.1)
    return fig, ax


def plot_polar(angles, spectrum, ax, log=False, angle_gt=None, ylabels=True, **plot_kwargs):
    from matplotlib.ticker import FormatStrFormatter
    
    assert spectrum.shape[0] == 1
    if log:
        spectrum_chosen = np.log10(spectrum[0])
    else:
        spectrum_chosen = spectrum[0]
    max_amplitude = np.max(spectrum_chosen)
    indices = np.where(spectrum_chosen == max_amplitude)[0]
    ax.plot(angles, spectrum_chosen, **plot_kwargs)
    [ax.axvline(angles[idx], **plot_kwargs) for idx in indices]
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2g'))
    
    if angle_gt is not None:
        ax.axvline(angle_gt, color="k", ls=":")

    plt.draw()
    
    yticklabels = ax.get_yticklabels()
    yticks = ax.get_yticks()
    if len(yticklabels) > 3:
        yticks = yticks[-5::2]
        yticklabels = yticklabels[-5::2]
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticklabels)
    ax.set_xticklabels([])
    if not ylabels:
        ax.set_yticklabels([])

In [ ]:
from generate_doa_results import (
    generate_signals_analytical,
    generate_signals_pyroom,
    get_source,
    from_0_to_2pi,
)

def get_buffers_single(mics, time, frequency_hz, simulation_type, noise, farfield):
    from crazyflie_description_py.parameters import N_BUFFER, FS
    
    time_index = int(2 * np.linalg.norm(sources[0]) * SPEED_OF_SOUND // FS)
    
    if simulation_type == "pyroom":
        signals_received = generate_signals_pyroom(
            sources,
            mics,
            frequency_hz,
            time,
            noise=noise,
            phase_offset=phase_offsets,
        )
    elif simulation_type == "analytical":
        time_index = 0
        signals_received = generate_signals_analytical(
            sources,
            mics,
            frequency_hz,
            time,
            noise=noise,
            phase_offset=phase_offsets,
        )
    else:
        raise ValueError(simulation_type)

    buffers = signals_received[:, time_index : time_index + N_BUFFER]
    return buffers

def get_buffers(mics_list, times_list, frequency_desired, simulation_type, noise=1e-3,
                farfield=False):
    """ 
    Simulate received buffers for given mic positions and corresponding times.
    
    :param mics_list: list of mic_positions
    :param times_list: list of times corresponding to mic_positions(seconds)
    :param frequency_desired: source frequency in Hz. 
    :param simulation_type: analytical or pyroom
    
    """
    freq_index, frequency_hz = get_frequency(frequency_desired)
    indices = [freq_index]

    #### moving mic array ####
    buffer_f_list = []
    for mics, time in zip(mics_list, times_list):
        buffers = get_buffers_single(mics, time, frequency_desired, simulation_type, noise, farfield)
        buffer_f = np.fft.rfft(buffers)[:, indices]
        buffer_f_list.append(buffer_f.T)
        
    #### multi-mic array ####
    mics_array = np.concatenate([*mics_list])
    
    time = times_list[0]
    buffer_multimic = get_buffers_single(mics_array, time, frequency_desired, simulation_type, noise, farfield)
    buffer_f_multimic = np.fft.rfft(buffer_multimic)[:, indices]
    return buffer_f_list, buffer_f_multimic

In [ ]:
def do_beamforming(source, mics, frequency_hz, noise=1e-3):
    """ Simplified beamforming function for theoretical analysis
    """
    from generate_doa_results import (
        generate_signals_analytical,
        generate_signals_pyroom,
    )
    from crazyflie_description_py.parameters import N_BUFFER
    from audio_stack.beam_former import BeamFormer
    
    signals = generate_signals_analytical(
        source, mics, frequency_hz, time=0, noise=noise, farfield=True
    )
    buffers = signals[:, :N_BUFFER]
    buffers_f = np.fft.rfft(buffers)

    beam_former = BeamFormer(mic_positions=mics)
    R = beam_former.get_correlation(buffers_f.T)[[freq_index], ...]

    if method == "das":
        spectrum = beam_former.get_das_spectrum(R, np.array([frequency_hz]))
    else:
        spectrum = beam_former.get_mvdr_spectrum(R, np.array([frequency_hz]))
    spectrum = (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))
    spectrum[spectrum < 1e-20] = 1e-20
    angles = BeamFormer.theta_scan
    return angles, spectrum


def get_mic_array(n_mic, baseline, method="uniform"):
    if method == "uniform":
        return np.array([[baseline * n, 0] for n in range(n_mic)])
    elif method == "circular":
        angle = 2 * np.pi / n_mic
        angles = angle * np.arange(n_mic)
        # makes sure spacing between two mics is always the same.
        radius = baseline / np.sqrt(2 * (1 - np.cos(angle)))
        mics = np.array(
            [
                [radius * np.cos(angles[n]), radius * np.sin(angles[n])]
                for n in range(n_mic)
            ]
        )
        mics[:, 0] += radius
        np.testing.assert_allclose(np.linalg.norm(mics[1, :] - mics[0, :]), baseline)
        return mics
    else:
        raise ValueError(method)

def get_baselines(frequency_hz, baseline_factors):
    from constants import SPEED_OF_SOUND
    baseline_limit = SPEED_OF_SOUND / frequency_hz / 2
    baseline = baseline_limit * 2.0 ** baseline_factors
    return baseline_limit, baseline

def get_source(angle_rad, source_distance=1, degrees=False):
    if np.ndim(angle_rad) == 0:
        if degrees:
            angle_rad = angle_rad / 180 * np.pi
        return source_distance * np.array((np.cos(angle_rad), np.sin(angle_rad)))
    else:
        if degrees:
            angle_rad = deepcopy(angle_rad)
            angle_rad = np.array(angle_rad) / 180 * np.pi
        return [get_source(a) for a in angle_rad]
    
def get_frequency(frequency_desired):
    from crazyflie_description_py.parameters import N_BUFFER, FS
    freqs = np.fft.rfftfreq(N_BUFFER, 1 / FS)
    freq_index = np.argmin(np.abs(freqs - frequency_desired))
    frequency_hz = freqs[freq_index]
    if frequency_hz != frequency_desired:
        print(f'using {frequency_hz}Hz instead of {frequency_desired}Hz')
    return freq_index, frequency_hz

# 1. Limits study

In [ ]:
frequency_desired = 1000 # in hz, will be changed to available frequency bin
source_distance = 2 # not actually used, since we use farfield assumption
signal_noise = 1e-3  # 1e-2

method = "das"; log = False
#method = "mvdr"; log=True

dir_name = "plots_doa/theory"

freq_index, frequency_hz = get_frequency(frequency_desired)

label = str.upper(method) if method == "das" else str.upper(method) + " (log)"

## 1.1 Beamform vs. baseline

In [ ]:
# ==== parameters for this experiment
baseline_factors = np.arange(-2, 3, step=1)
angle_rad = 180 * np.pi / 180

# ==== usual stuff
source = get_source(angle_rad, source_distance=source_distance)
baseline_limit, baselines = get_baselines(frequency_hz, baseline_factors)

# ==== run experiment
fig, axs = plt.subplots(
    1, len(baselines), subplot_kw={"projection": "polar"}, sharey=True
)
fig.set_size_inches(3 * len(baselines), 3)
for i, baseline in enumerate(baselines):
    ax = axs[i]

    mics = get_mic_array(2, baseline, method="uniform")
    
    angles, spectrum = do_beamforming(source, mics, frequency_hz, noise=signal_noise)

    plot_polar(angles, spectrum, ax, log, color=f"C{i}")
    ax.set_title(f"$b= {baseline/baseline_limit} b_f$")
    ax.grid("minor")

axs[0].set_ylabel(label)
save_fig(fig, f"{dir_name}/baselines_{method}.pdf")

## 1.2 Beamform vs. number of mics

### Grow array

In [ ]:
# ==== parameters for this experiment
baseline_factor = -1
angle_rad = 120 * np.pi / 180
n_mics = 2 ** np.arange(1, 6)

# ==== usual stuff
source = get_source(angle_rad, source_distance=source_distance)
baseline_limit, baseline = get_baselines(frequency_hz, baseline_factor)

# ==== run experiment
ys = np.linspace(-baseline, baseline, len(n_mics))[::-1]
for array in ["circular", "uniform"]:
    fig_pos, ax_pos = plt.subplots(subplot_kw={"frameon": False})

    xmax = 0
    ymax = 0
    # ==== run experiment
    fig, axs = plt.subplots(
        1, len(n_mics), subplot_kw={"projection": "polar"}, sharey=True
    )
    fig.set_size_inches(3 * len(n_mics), 3)
    for i, n_mic in enumerate(n_mics):
        ax = axs[i]

        mics = get_mic_array(n_mic, baseline, method=array)

        if array == "uniform":
            ax_pos.scatter(mics[:, 0], mics[:, 1] + ys[i])
        elif array == "circular":
            ax_pos.scatter(mics[:, 0], mics[:, 1])

        xmax = max(np.max(mics[:, 0]), xmax)
        ymax = max(np.max(mics[:, 1]), ymax)

        angles, spectrum = do_beamforming(source, mics, frequency_hz)
        # spectrum = (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))

        plot_polar(angles, spectrum, ax, log, color=f"C{i}")
        ax.set_title(f"$N={n_mic}$")
    axs[0].set_ylabel(f"{label}, {array}")

    xlim = [-baseline, xmax + baseline]
    ylim = [-baseline * 7, baseline * 7]
    fig_pos.set_size_inches(5 * (xlim[1] - xlim[0]) / (ylim[1] - ylim[0]), 5)
    ax_pos.yaxis.set_visible(False)
    ax_pos.xaxis.set_visible(False)
    ax_pos.set_xlim(xlim)
    ax_pos.set_ylim(ylim)
    ax_pos.set_title(array)

    save_fig(fig_pos, f"{dir_name}/number_mics_grow_geometry_{array}.pdf")
    save_fig(fig, f"{dir_name}/number_mics_grow_{method}_{array}.pdf")

### Grow number

In [ ]:
# ==== parameters for this experiment
baseline_factor = -1
angle_rad = 120 * np.pi / 180
n_mics = 2 ** np.arange(1, 6)
noise = 1  # 20: ok 100: breaks down

# ==== usual stuff
source = get_source(angle_rad)
baseline_limit, baseline = get_baselines(frequency_hz, baseline)

ys = np.linspace(-baseline, baseline, len(n_mics))[::-1]
for array in ["circular", "uniform"]:

    fig_pos, ax_pos = plt.subplots(subplot_kw={"frameon": False})
    xmax = 0
    ymax = 0

    # ==== run experiment
    fig, axs = plt.subplots(
        1, len(n_mics), subplot_kw={"projection": "polar"}, sharey=True
    )
    fig.set_size_inches(3 * len(n_mics), 3)
    for i, n_mic in enumerate(n_mics):
        ax = axs[i]

        if array == "uniform":
            baseline_here = 2 * baseline / (n_mic - 1)
            mics = get_mic_array(n_mic, baseline_here, method=array)
            ax_pos.scatter(mics[:, 0], mics[:, 1] + ys[i])
        elif array == "circular":
            baseline_here = baseline * np.sqrt(2 * (1 - np.cos(2 * np.pi / n_mic)))
            mics = get_mic_array(n_mic, baseline_here, method=array)
            ax_pos.scatter(mics[:, 0], mics[:, 1] + ys[i])

        xmax = max(np.max(mics[:, 0]), xmax)
        ymax = max(np.max(mics[:, 1]), ymax)

        angles, spectrum = do_beamforming(source, mics, frequency_hz, noise=signal_noise)

        plot_polar(angles, spectrum, ax, log, color=f"C{i}")
        ax.set_title(f"$N={n_mic}$")
    axs[0].set_ylabel(f"{label}, {array}")

    xlim = [-baseline, xmax + baseline]
    ylim = [-baseline * 2, baseline * 2]
    fig_pos.set_size_inches(5 * (xlim[1] - xlim[0]) / (ylim[1] - ylim[0]), 5)
    ax_pos.yaxis.set_visible(False)
    ax_pos.xaxis.set_visible(False)
    ax_pos.set_xlim(xlim)
    ax_pos.set_ylim(ylim)
    ax_pos.set_title(array)

    save_fig(fig_pos, f"{dir_name}/number_mics_geometry_{array}.pdf")
    save_fig(fig, f"{dir_name}/number_mics_{method}_{array}.pdf")

## 1.3 Beamform vs. angle

In [ ]:
# ==== parameters for this experiment
baseline_factor = -1
angle_rads = np.arange(90, 181, step=22.5)[:5] * np.pi / 180
n_mic = 3

# ==== usual stuff
baseline_limit, baseline = get_baselines(frequency_hz, baseline_factor)

# ==== run experiment
for array in ["circular", "uniform"]:
    mics = get_mic_array(n_mic, baseline, method=array)

    fig, axs = plt.subplots(
        1, len(angle_rads), subplot_kw={"projection": "polar"}, sharey=True
    )
    fig.set_size_inches(3 * len(angle_rads), 3)
    for i, angle_rad in enumerate(angle_rads):
        ax = axs[i]

        source = get_source(angle_rad)
        angles, spectrum = do_beamforming(source, mics, frequency_hz)

        plot_polar(angles, spectrum, ax=ax, log=log, color=f"C{i}")
    axs[0].set_ylabel(f"{str.upper(method)}, {array}")
    save_fig(fig, f"{dir_name}/directions_{method}_{array}.pdf")

# 2. Dynamic DOA study

In [ ]:
from generate_doa_results import COMBINATION_METHOD, NORMALIZATION_METHOD

def kinetic_doa(plot=False, extension=".pdf"):
    print("using", method)
    frequencies = np.array([frequency_hz])

    beam_former = BeamFormer(mic_positions=mics_local)
    beam_former.init_multi_estimate(frequencies, combination_n=len(degrees))
    beam_former.init_dynamic_estimate(
        frequencies,
        combination_n=len(degrees),
        combination_method=COMBINATION_METHOD,
        normalization_method=NORMALIZATION_METHOD,
    )

    ax = None

    # note that we use degree, the theoretical orientation. the drone has actually
    # moved to degree_noisy (which is used in get_buffers)
    for (sig_f, time, degree, offset) in zip(buffer_f_list, times_list_noisy, degrees, offsets):
        beam_former.add_to_multi_estimate(sig_f, frequencies, time, degree, offset)
        spec = beam_former.add_signals_to_dynamic_estimates(
            sig_f, frequencies, degree, method=method
        )

        if plot:
            fig, ax = plot_spectrum(spec, polar=True, title="raw spectra", log=log, ax=ax, ylabels=False,
                                    mics_clean=mics_clean, sources=sources)
            fig.set_size_inches(3, 3)

    spectrum_dynamic = beam_former.get_dynamic_estimate()
    spectrum_delayed = beam_former.get_multi_estimate(method=method)

    if plot:
        save_fig(fig, f"{dir_name}/raw{extension}")

        fig, ax = plot_spectrum(spectrum_dynamic, polar=True, title="angle shift", log=log, ylabels=False,
                                mics_clean=mics_clean, sources=sources)
        fig.set_size_inches(3, 3)
        save_fig(fig, f"{dir_name}/angle{extension}")

        fig, ax = plot_spectrum(spectrum_delayed, polar=True, title="time shift", log=log, ylabels=False,
                                mics_clean=mics_clean, sources=sources)
        fig.set_size_inches(3, 3)
        save_fig(fig, f"{dir_name}/time{extension}")
        
def separate_doa(plot=False, polar=True):
    # note that we use degree, the theoretical orientation. the drone has actually
    # moved to degree_noisy (which is used in get_buffers)
    for buffer_f, mics, degree in zip(buffer_f_list, mics_clean, degrees):
        beam_former = BeamFormer(mic_positions=mics)
        R = beam_former.get_correlation(buffer_f)

        if method == "das":
            spectrum = beam_former.get_das_spectrum(R, np.array([frequency_hz]))
        else:
            spectrum = beam_former.get_mvdr_spectrum(
                R, np.array([frequency_hz]), inverse="pinv"
            )
        if plot:
            fig, ax = plot_spectrum(spectrum, polar=True, title=f"drone rotation {degree}deg", log=log, ylabels=False, 
                                    mics_clean=mics_clean, sources=sources)
            fig.set_size_inches(3, 3)
                
def multi_mic_doa(plot=False, polar=True, extension=".pdf"):
    # note that we use the clean mics here, although the signals are actually
    # coming from the noisy mics_list. 
    mics_array = np.concatenate([*mics_clean])
    beam_former = BeamFormer(mics_array)
    R_multimic = beam_former.get_correlation(buffer_f_multimic.T)
    frequencies = np.array([frequency_hz])

    if method == "das":
        print("using das")
        spectrum_multimic = beam_former.get_das_spectrum(R_multimic, frequencies)
    else:
        print("using mvdr")
        spectrum_multimic = beam_former.get_mvdr_spectrum(R_multimic, frequencies)

    if plot:
        fig, ax = plot_spectrum(
            spectrum_multimic, polar=True, title=f"{mics_array.shape[0]}-microphone array", log=log,
            ylabels=False, mics_clean=mics_clean, sources=sources
        )
        fig.set_size_inches(3, 3)
        save_fig(fig, f"{dir_name}/multimic{extension}")

        
def plot_setup(degrees, mics_list, sources):
    fig = plt.figure()
    fig.set_size_inches(3, 3)
    for i, (degree, mics) in enumerate(zip(degrees, mics_list)):
        mics_cm = mics * 1e2
        plt.scatter(*mics_cm.T, label=f"{degree:.0f}deg", color=f"C{i}", marker="x")
        
        center = np.mean(mics_cm, axis=0)
        mic_radius = 2 * np.max(np.linalg.norm(mics_cm - center, axis=1))
        [plt.plot([center[0], mic[0]], 
                  [center[1], mic[1]], 
                  color=f"C{i}", ls=":") for mic in mics_cm]
                          
        for source in sources:
            source_cm = source * 1e2
            normal = (source_cm - center) / np.linalg.norm(source_cm - center)
            plt.arrow(
                center[0],
                center[1],
                mic_radius * normal[0],
                mic_radius * normal[1],
                color="k",
                width=0.1,
            )
            # label='source')
    #l = plt.legend(
    #    bbox_to_anchor=[-0.3, 1],
    #    loc="lower left",
    #    title="mic rotations",
    #    ncol=len(degrees),
    #)
    plt.axis("equal")
    plt.xlabel("x [cm]")
    plt.ylabel("y [cm]")
    # plt.gca().add_artist(l)
    # plt.title('simulation setup')
    # plt.legend(loc='upper left')
    return fig

## 2.1 Resolve high frequency

In [ ]:
# constants 
from utils.geometry import Context
from generate_doa_results import get_movement

frequency_desired = 4000 # in hz, will be changed to available frequency bin
signal_noise = 1e-3  # 1e-2
simulation_type = "analytical"
#simulation_type = "pyroom"

source_distance = 1 # in meters
freq_index, frequency_hz = get_frequency(frequency_desired)

method = "das"; log = False
#method = "mvdr"; log=True

dir_name = "plots_doa/simulation"

label = str.upper(method)
if log:
    label += " (log)"
    
context = Context.get_crazyflie_setup()
mics_local = context.mics
mics_local -= np.mean(mics_local, axis=0)  # center the drone

In [ ]:
gt_angles_deg = [130]  # degrees, angle of source
phase_offsets = [0.0]  # phase offsets of each source

angular_velocity_deg = 30  # deg/sec, angular velocity of drone
linear_velocity_cm = 0 # cm/sec, linear velocity
sampling_time = 0.5 # was 1.0
n_samples = 3 # was 2

degree_noise = 0 #1 # 5 # noise added to each movement, in degrees
offset_noise = 0 #1e-2 # in m
time_quantization = 6  # number of decimal places to keep
time_noise = 0 #1e-6 #1e-6  # 1e-6 # noise added to recording times

In [ ]:
from audio_stack.beam_former import rotate_mics
from generate_doa_results import move_mics, get_noisy_times

sources = get_source(gt_angles_deg, degrees=True)
degrees, offsets, times_list = get_movement(angular_velocity_deg, linear_velocity_cm, n_samples, sampling_time)

# create noisy versions
# what we actually moved to
mics_list = move_mics(mics_local, degrees, offsets, {"degree": degree_noise, "offset": offset_noise})
# what we think we moved to
mics_clean = move_mics(mics_local, degrees, offsets)
# what we think we measured at
times_list_noisy = get_noisy_times(times_list, time_noise)

print('movement of first mic:', np.linalg.norm(mics_clean[0][0] - mics_clean[1][0]))
mics_array = np.concatenate([*mics_clean])
print('baselines individual: \n', np.linalg.norm(mics_local[None, ...] - mics_local[:, None, :], axis=2)[0,:])
print('baselines combined: \n', np.linalg.norm(mics_array[None, ...] - mics_array[:, None, :], axis=2)[0,:])
baseline_limit = SPEED_OF_SOUND / frequency_hz / 2
print('baseline limit:', baseline_limit)

fig = plot_setup(degrees, mics_list, sources)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
save_fig(fig, f"{dir_name}/setup_noiseless_one.pdf")
pass

In [ ]:
# sanity checks
for mics, time in zip(mics_list, times_list):
    fig, ax = plt.subplots()

    buffers = np.zeros((mics.shape[0], N_BUFFER))
    for gt_angle_rad in gt_angles_rad:
        
        source = get_source(gt_angle_rad) 
        
        signals_received_an = generate_signals_analytical(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax
        )
        signals_received_py = generate_signals_pyroom(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax
        )

        if simulation_type == "pyroom":
            buffers += signals_received_py[:, time_index : time_index + N_BUFFER]
        elif simulation_type == "analytical":
            buffers += signals_received_an[:, time_index : time_index + N_BUFFER]
        else:
            raise ValueError(simulation_type)

    ax.set_xlim(1300, 1400)
    ax.legend()
    ax.set_title(f"signals at time {time:.3f}")

    fig, axs = plt.subplots(buffers.shape[0])
    n_plot = 50
    fig.suptitle(f"first {n_plot} samples of received buffers")
    for i, ax in enumerate(axs):
        ax.plot(buffers[i, :n_plot], color=f"C{i}")

In [ ]:
# note that we use mics_list here, which is the noisy version.
buffer_f_list, buffer_f_multimic = get_buffers(
    mics_list, times_list, frequency_hz, simulation_type, noise=signal_noise
)

In [ ]:
from audio_stack.beam_former import BeamFormer
separate_doa(plot=True)

In [ ]:
multi_mic_doa(plot=True, extension=f"_noiseless_one_{method}.pdf")

In [ ]:
kinetic_doa(plot=True, extension=f"_noiseless_one_{method}.pdf")

## 2.2 Multiple source test

In [ ]:
from generate_doa_results import move_mics, get_noisy_times
np.random.seed(10)

frequency_desired = 4000
freq_index, frequency_hz = get_frequency(frequency_desired)

source_type = "uniform2"  # 2, 3 good, 4 ok, 5 broken
#source_type = "close40" # 10, 20 no, 30 ok, 40 good

angular_velocity_deg = 30  # deg/sec, angular velocity of drone
linear_velocity_cm = 0 # cm/sec, linear velocity
sampling_time = 0.5
n_samples = 4

degree_noise = 0 #1 # 5 # noise added to each movement, in degrees
offset_noise = 0 #1e-2 # in m
time_quantization = 6  # number of decimal places to keep
time_noise = 0 #1e-6  # 1e-6 # noise added to recording times

degrees, offsets, times_list = get_movement(angular_velocity_deg, linear_velocity_cm, n_samples, sampling_time)

mics_list = move_mics(mics_local, degrees, offsets, {"degree": degree_noise, "offset": offset_noise})
mics_clean = move_mics(mics_local, degrees, offsets)
times_list_noisy = get_noisy_times(times_list, time_noise)

for source_type in [f"uniform{i}" for i in range(2, 8)]:
    n = int(source_type.split("uniform")[-1])
    gt_angles_deg = np.arange(10, 360, step=360/n)
    gt_angles_deg += np.random.normal(scale=5)
    
    phase_offsets = np.random.uniform(0, 2 * np.pi, size=len(gt_angles_deg))

    sources = get_source(gt_angles_deg, degrees=True)
    fig = plot_setup(degrees, mics_list, sources)
    save_fig(fig, f"{dir_name}/setup_{source_type}.pdf")

    buffer_f_list, buffer_f_multimic = get_buffers(
        mics_list, times_list, frequency_hz, simulation_type, noise=signal_noise
    )
    multi_mic_doa(plot=True, extension=f"_{source_type}_{method}.pdf")
    kinetic_doa(plot=True, extension=f"_{source_type}_{method}.pdf")

In [ ]:
from generate_doa_results import move_mics, get_noisy_times
np.random.seed(10)

frequency_desired = 4000
freq_index, frequency_hz = get_frequency(frequency_desired)

source_type = "uniform2"  # 2, 3 good, 4 ok, 5 broken
#source_type = "close40" # 10, 20 no, 30 ok, 40 good

angular_velocity_deg = 30  # deg/sec, angular velocity of drone
linear_velocity_cm = 10 # cm/sec, linear velocity
sampling_time = 0.5
n_samples = 4

degree_noise = 1 #1 # 5 # noise added to each movement, in degrees
offset_noise = 1e-2 #1e-2 # in m
time_quantization = 6  # number of decimal places to keep
time_noise = 0 #1e-6  # 1e-6 # noise added to recording times

degrees, offsets, times_list = get_movement(angular_velocity_deg, linear_velocity_cm, n_samples, sampling_time)

mics_list = move_mics(mics_local, degrees, offsets, {"degree": degree_noise, "offset": offset_noise})
mics_clean = move_mics(mics_local, degrees, offsets)
times_list_noisy = get_noisy_times(times_list, time_noise)

for source_type in [f"close{i}" for i in [10, 30, 50]]:
    delta = int(source_type.split("close")[-1])
    gt_angles_deg = [100, 100+delta]  # , 180, 270]
    phase_offsets = np.random.uniform(0, 2 * np.pi, size=len(gt_angles_deg))

    ################################

    sources = get_source(gt_angles_deg, degrees=True)
    fig = plot_setup(degrees, mics_list, sources)
    save_fig(fig, f"{dir_name}/setup_{source_type}.pdf")

    buffer_f_list, buffer_f_multimic = get_buffers(
        mics_list, times_list, frequency_hz, simulation_type, noise=signal_noise
    )
    multi_mic_doa(plot=True, extension=f"_{source_type}_{method}.pdf")
    kinetic_doa(plot=True, extension=f"_{source_type}_{method}.pdf")

## 2.3 Test with lateral movement

In [ ]:
from generate_doa_results import move_mics, get_noisy_times

frequency_desired = 4000
freq_index, frequency_hz = get_frequency(frequency_desired)

gt_angles_deg = [50]  # degrees, angle of source
phase_offsets = [0.0]  # phase offsets of each source

angular_velocity_deg = 0  # deg/sec, angular velocity of drone
sampling_time = 0.5
n_samples = 3

degree_noise = 0 # 5 # noise added to each movement, in degrees
offset_noise = 0 # in m
time_quantization = 6  # number of decimal places to keep
time_noise = 0  # 1e-6 # noise added to recording times

sources = get_source(gt_angles_deg, degrees=True)

for vel in [4, 10, 30, 50]:
    linear_velocity_cm = vel # cm/sec, linear velocity # 50 broken
    degrees, offsets, times_list = get_movement(angular_velocity_deg, linear_velocity_cm, n_samples, sampling_time)

    # create noisy versions
    mics_list = move_mics(mics_local, degrees, offsets, {"degree": degree_noise, "offset": offset_noise})
    mics_clean = move_mics(mics_local, degrees, offsets)
    times_list_noisy = get_noisy_times(times_list, time_noise)

    fig = plot_setup(degrees, mics_list, sources)
    save_fig(fig, f"{dir_name}/setup_lateral{linear_velocity_cm}.pdf")

    buffer_f_list, buffer_f_multimic = get_buffers(
        mics_list, times_list, frequency_hz, simulation_type, noise=signal_noise
    )
    multi_mic_doa(plot=True, extension=f"_lateral{linear_velocity_cm}_{method}.pdf")
    kinetic_doa(plot=True, extension=f"_lateral{linear_velocity_cm}_{method}.pdf")

# 3. Full pipeline

In [ ]:
def get_angle_error(spec, error_type="success"):
    def unwrap_and_deg(est_deg, gt_deg):
        unwrap_angles = np.unwrap(np.array([est_deg, gt_deg]) / 180 * np.pi)
        return abs(unwrap_angles[1] - unwrap_angles[0]) * 180 / np.pi
    
    if type(GT_ANGLE_DEG) == list:
        errors = []
        
        peaks, *_ = find_peaks(spec)
        angles_est = angles[peaks[:len(GT_ANGLE_DEG)]]
        for gt_angle_deg in GT_ANGLE_DEG:
            
            if error_type == "success":
                if np.any(np.abs(angles_est - gt_angle_deg) < 5):
                    errors.append(1)
                else:
                    errors.append(0)
            
            elif error_type == "abs":
                # find the closest peak
                est_angle_deg = angles_est[np.argmin(np.abs(gt_angle_deg - angles_est))]
                errors.append(unwrap_and_deg(est_angle_deg, gt_angle_deg))
        return np.mean(errors)
    else:
        est_angle_deg = angles[np.argmax(spec)]
        return unwrap_and_deg(est_angle_deg, GT_ANGLE_DEG)


## result overview

In [ ]:
import pandas as pd
from scipy.signal import find_peaks

label_list = ["spectrum_dynamic", "spectrum_delayed", "spectrum_multimic"]

#GT_ANGLE_DEG = 50
#fname = 'results/doa_lateral_movement.pkl'; plot_by = "linear_velocity_cm"; unit="cm/s"

#GT_ANGLE_DEG = 130
#fname = 'results/doa_degree_noise.pkl'; plot_by = "degree_noise"; unit="deg"

GT_ANGLE_DEG = [10, 50]
fname = 'results/doa_multi.pkl'; plot_by = "degree_noise"; unit="deg"

#fname = 'results/doa_time_noise.pkl'; plot_by = "time_noise"; unit="s"
df = pd.read_pickle(fname)

frequency = 5000

df = pd.read_pickle(fname)
freq_i = 0  # only have one frequency for each experiment.

n_angles = df.iloc[0].spectrum_multimic.shape[1]  # n_freq x n_angles
angles = np.linspace(0, 360, n_angles)

noise_levels = df[plot_by].unique()
n_noise_levels = len(noise_levels)
print("number of noise levels (=rows):", n_noise_levels)
n_it = len(df.it.unique())
boxplot_mat = np.empty(
    (len(label_list), n_it, n_noise_levels)
)  # each slice will be one boxplot

for d, (noise_level, df_noise_level) in enumerate(df.groupby(plot_by)):
    df_noise_level = df_noise_level.loc[df_noise_level.frequency==frequency]
    fig, axs = plt.subplots(1, len(label_list))  # sharey=True, sharex=True)
    fig.suptitle(f"{plot_by.replace('_', ' ')} {noise_level:.2g}{unit}", y=1.1)
    fig.set_size_inches(10, 3)
    
    for counter, (__, row) in enumerate(df_noise_level.iterrows()):
        for l, label in enumerate(label_list):
            if counter <= 3:
                spec = row[label][freq_i]
                spec = (spec - np.min(spec)) / (np.max(spec) - np.min(spec))
                axs[l].semilogy(angles, spec, label=f"it{counter}", color=f"C{counter}")
                # axs[l].set_ylim(1e-9, 2)
                
                peaks, *_ = find_peaks(spec)
                [axs[l].axvline(angles[p], color=f"C{counter}") for p in peaks[:2]]

            # calculate angle estimate
            error_deg = get_angle_error(row[label][freq_i])
            boxplot_mat[l, counter, d] = error_deg
            #print(f"error for {counter}, {label}, noise level {noise_level}", error_deg)

    for label, ax in zip(label_list, axs):
        if type(GT_ANGLE_DEG) == list:
            [ax.axvline(g, color="black", ls=":") for g in GT_ANGLE_DEG]
        else:
            ax.axvline(GT_ANGLE_DEG, color="black", ls=":")
        ax.set_xlabel("angle [deg]")
        ax.set_title(label)
    axs[0].set_ylabel("spectrum [-]")

In [ ]:
fig_box, ax_box = plt.subplots(1, len(label_list), sharex=True, sharey=True)
fig_box.set_size_inches(10, 5)
for l, label in enumerate(label_list):
    ax_box[l].boxplot(boxplot_mat[l], positions=range(len(noise_levels)), widths=0.8)
    ax_box[l].set_xticklabels([f"{n:.1g}" for n in noise_levels], rotation=90)
    ax_box[l].set_title(label)
    #ax_box[l].set_yscale('log')
    ax_box[l].set_xlabel(f"{plot_by.replace('_', ' ')}[{unit}]")
ax_box[0].set_ylabel("absolute error [deg]")

## heatmaps

In [ ]:
def plot_pivot_table(df, column="time_noise", index="frequency", 
                     values="spectrum_delayed", log=False, labels=["mean"], saveas="",
                     xlabel=None, vmax=None, title=None):

    from utils.plotting_tools import pcolorfast_custom, add_colorbar

    def aggfunc(column, function=np.mean):
        values = []
        for row in column:
            error_deg = get_angle_error(row[freq_i])
            values.append(error_deg)
        return function(values)

    def mean(column):
        return aggfunc(column, np.mean)

    def std(column):
        return aggfunc(column, np.std)


    #mean = lambda x: aggfunc(x, np.mean)
    #std = lambda x: aggfunc(x, np.std)
    tb = pd.pivot_table(data=df, values=values, index=index, 
                        columns=column,
                        aggfunc=[mean, std]) #{"mean": mean,  "std": std})

    for label in labels:
        tb_mean = tb.iloc[:, tb.columns.get_level_values(0)==label]
        noises = tb_mean.columns.get_level_values(1)
        
        if log:
            values = np.log10(tb_mean.values)
        else:
            values = tb_mean.values

        fig, ax = plt.subplots()
        fig.set_size_inches(5, 4)
        im = pcolorfast_custom(ax, np.arange(len(noises)), tb_mean.index.values, values, vmax=vmax)
        add_colorbar(fig, ax, im)
        
        yticks = tb_mean.index.values
        yticks_rounded = np.unique(yticks // 1000 * 1000)
        ax.set_yticks(yticks_rounded + (yticks[1]-yticks[0])/2)
        ax.set_yticklabels(yticks_rounded)
        
        if max(noises) <= 1.0:
            if len(noises) > 10:
                ax.set_xticks(np.arange(len(noises), step=2)+0.5)
                ax.set_xticklabels(np.round(np.log10(noises), 1)[::2])
            else:
                ax.set_xticklabels(np.round(np.log10(noises), 1))
            ax.set_xlabel(column.replace("_", " ") + " (log)")
        else:
            if len(noises) > 10:
                ax.set_xticks(np.arange(len(noises), step=2)+0.5)
                ax.set_xticklabels(noises[::2])
            else:
                ax.set_xticklabels(noises)
            ax.set_xlabel(column.replace("_", " "))
        if xlabel is not None:
            ax.set_xlabel(xlabel)
        
        if index == "frequency":
            ax.set_ylabel("frequency [Hz]")
        else:
            ax.set_ylabel(index.replace("_", " "))
            
        if title is None:
            ax.set_title(label + " of angle error [$^\\circ$]")
        else:
            ax.set_title(title)
        
        if saveas != "":
            save_fig(fig, f"plots_doa/simulation/{saveas}_{label}.pdf")

In [ ]:
GT_ANGLE_DEG = [10, 50]
fname = 'results/doa_multi.pkl' # was done using angular movement only
df = pd.read_pickle(fname)
df.frequency = df.frequency.astype(int)
for spectrum in ["spectrum_dynamic", "spectrum_delayed", "spectrum_multimic"]:
    plot_pivot_table(df, column="degree_noise", 
                     values=spectrum, log=False, saveas=f"multi_{spectrum}", 
                     title="success rate", 
                     xlabel="angle noise $\\sigma_\\alpha$ [$^\\circ$]")

In [ ]:
GT_ANGLE_DEG = [10, 50]
fname = 'results/doa_multi_joint_highres.pkl' # done using angular and linear movement
df = pd.read_pickle(fname)
df.frequency = df.frequency.astype(int)
for spectrum in ["spectrum_dynamic", "spectrum_delayed", "spectrum_multimic"]:
    plot_pivot_table(df, column="degree_noise", 
                     values=spectrum, log=False, saveas=f"multi_joint_{spectrum}", 
                     title="success rate", 
                     xlabel="angle noise $\\sigma_\\alpha$ [$^\\circ$]")

In [ ]:
GT_ANGLE_DEG = 50
#fname = 'results/doa_lateral_movement_highres.pkl'
fname = 'results/doa_lateral_movement.pkl'
new_label = "delta [cm]" 
df = pd.read_pickle(fname)
df.frequency = df.frequency.astype(int)
df.loc[:, new_label] = df.linear_velocity_cm * 0.5
df.loc[:, new_label] = df.loc[:, new_label].astype(np.int)
for spectrum in ["spectrum_dynamic", "spectrum_delayed", "spectrum_multimic"]:
    plot_pivot_table(df, column=new_label, values=spectrum, log=False, saveas=f"lateral_movement_{spectrum}", vmax=100)

In [ ]:
GT_ANGLE_DEG = 130
fname = 'results/doa_time_noise.pkl'
df = pd.read_pickle(fname)
df.frequency = df.frequency.astype(int)
plot_pivot_table(df, column="time_noise", saveas="time_noise", xlabel="time noise $\\sigma_t$ (log) [s]", log=True, 
                 title="mean of angle error (log) [$^\\circ$]")#90 log=True)

In [ ]:
GT_ANGLE_DEG = 130
for spectrum in ["spectrum_dynamic", "spectrum_delayed", "spectrum_multimic"]:
    fname = 'results/doa_degree_noise.pkl'
    df = pd.read_pickle(fname)
    plot_pivot_table(df, values=spectrum, column="degree_noise", saveas=f"degree_noise_{spectrum}",
                     xlabel="angle noise $\\sigma_\\alpha$ [$^\\circ$]", vmax=2, log=True, 
                     title="mean of angle error (log) [$^\\circ$]")#90 log=True)